# 🛡️ Arquitectura del Modelo de Detección de Anomalías: Enfoque Híbrido
Para abordar el reto de identificar irregularidades en 31 millones de registros inmobiliarios sin contar con etiquetas previas de fraude (aprendizaje no supervisado), se diseñó una arquitectura de detección híbrida. Esta combina la rigidez de las reglas de negocio forenses con la flexibilidad de la Inteligencia Artificial.

**1. Ingeniería de Características (Feature Engineering)**
La selección de variables no fue arbitraria; se basó en hipótesis de fraude inmobiliario y calidad de datos. Se transformaron variables crudas en indicadores de riesgo matemático:

**A. Variables Base y su Justificación de Riesgo**
- **VALOR (Dimensión Financiera):** Variable crítica. Los valores extremos indican dos tipologías de fraude opuestas: Lavado de Activos (sobrevaloración) o Evasión Fiscal/Ventas Simbólicas (subvaloración).

- **FECHA_APERTURA_TEXTO (Dimensión de Calidad):** La ausencia de esta fecha rompe la trazabilidad histórica del predio, lo cual es una "Red Flag" operativa y un posible indicador de manipulación de registros antiguos.

- **PREDIOS_NUEVOS (Dimensión de Urbanismo):** Los predios recién matriculados presentan mayor riesgo de urbanización ilegal, "volteo de tierras" o inconsistencias en la tradición registral.

- **COUNT_DE y COUNT_A (Dimensión de Red/Complejidad):** Mide la fragmentación de la propiedad. Un número inusualmente alto de otorgantes (vendedores) o adquirientes (compradores) sugiere loteo ilegal, procesos de falsa tradición o testaferrato.

- **YEAR_RADICA (Dimensión Temporal):** Permite aislar el efecto inflacionario y detectar patrones anómalos específicos en periodos electorales o cambios de administración.

- **DIVIPOLA (Contexto Geográfico):** Fundamental para la normalización. Permite comparar precios dentro de su mercado local (ej. no comparar precios de Bogotá con Leticia).


**B. Transformaciones Avanzadas (El "Truco" Matemático)**
Para que el modelo matemático fuese robusto frente a la enorme variabilidad de precios en Colombia, se generaron tres características sintéticas:

1. **`FECHA_APERTURA_INCONSISTENTE` (Binaria):**

    - _Lógica:_ Flag (1/0) que penaliza explícitamente la falta de integridad en la data temporal.

2. **`VALOR_LOG` (Normalización):**

    - _Lógica:_ Se aplicó una transformación logarítmica (log1p) al valor de la transacción. Esto comprime la escala de precios, permitiendo que el modelo analice simultáneamente transacciones de interés social y megaproyectos sin que los valores extremos sesguen el aprendizaje.

3. **`Z_SCORE_RELATIVO` (Contextualización Geoespacial):**

    - _La Joya de la Corona:_ Calculamos cuántas desviaciones estándar se aleja el precio de una transacción respecto al promedio de su Municipio (DIVIPOLA).

    - _Impacto:_ Nos permite detectar la transacción más cara de un pueblo pobre (posible lavado) o la más barata de un barrio rico (evasión), haciéndolas comparables matemáticamente.


**2. Estrategia de Detección: El Modelo Híbrido**
Dado que las anomalías inmobiliarias son heterogéneas (algunas son errores de dedo, otras son esquemas complejos), un solo algoritmo no es suficiente. Implementamos un sistema de Defensa en Profundidad con tres capas:

**Capa 1: Reglas de Negocio Forenses (Hard Rules)**
Detecta lo obvio y lo inadmisible. Se establecieron umbrales basados en percentiles extremos (P01 y P99) y lógica de mercado.

   - **Detección de Subvaloración Crítica:** Transacciones con valores inferiores a $100.000 COP (Umbral P01), indicativas de error de calidad o simulación de venta.

   - **Detección de Sobreprecio Extremo:** Transacciones que superan las 3.5 desviaciones estándar (Z-Score > 3.5) respecto a su mercado local.

**Capa 2: Inteligencia Artificial (Isolation Forest)**
Detecta lo sutil y estructural. Se utilizó el algoritmo Isolation Forest (entrenado con contamination calculada de acuerdo al Z-score sobre las características transformadas).

   - **Objetivo:** Identificar patrones multidimensionales que escapan a las reglas simples, como combinaciones atípicas entre número de intervinientes, antigüedad del predio y valor.

   - **Funcionamiento:** El modelo aísla observaciones "raras" en el espacio vectorial; aquellas que requieren menos cortes para ser aisladas se marcan como anómalas.

**Capa 3: Consolidación y Taxonomía**
El resultado final no es binario, sino descriptivo. Se fusionan los hallazgos de las capas anteriores para etiquetar cada anomalía con su causa raíz, facilitando la auditoría:

   - `Sobreprecio extremo` / `Subvaloración extrema`

   - `Patrón atípico IA` (Detectado exclusivamente por el algoritmo)

**Resultado:** Un sistema robusto que minimiza falsos positivos al contextualizar los precios, pero maximiza la detección de fraudes complejos gracias al aprendizaje de máquina.

# Preparación del entorno

In [28]:
# Necesario para importar módulos desde el directorio raíz

import sys
import os

ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(ROOT)

In [54]:
# Third-party imports
import numpy as np
import pandas as pd
import polars as pl

from sklearn.ensemble import IsolationForest

# Local application imports
from src.data_loader import obtener_datos_polars

# Obtención de datos

In [ ]:
# Se obtienen los datos que fueron previamente procesados en Databricks

sql = """
SELECT *
FROM default.data_modelo_anomalias_transaccionales
"""

datos_modelo = obtener_datos_polars(sql)

datos_modelo = datos_modelo.to_pandas()  # Convertir a pandas DataFrame para cargar al modelo

d:\Hackathon-MinTIC-2025\src\data_loader.py:24: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  df = pl.DataFrame(data=filas, schema=columnas)


In [ ]:
# Vistazo rápido a los datos

datos_modelo.head()

,PK,VALOR,YEAR_RADICA,PREDIOS_NUEVOS,COUNT_DE,COUNT_A,FECHA_APERTURA_INCONSISTENTE,VALOR_LOG,Z_SCORE_RELATIVO
0,11001-50C-007144-00024-00125-2023,280000000.0,2023,0,1,1,0,19.450300,0.612230
1,11001-50C-010059-00021-00125-2023,269000000.0,2023,0,1,1,0,19.410222,0.588895
2,11001-50C-1076203-00009-00125-2022,103000000.0,2022,0,1,1,0,18.450240,0.029950
3,11001-50C-1127133-00012-00125-2021,180000000.0,2021,0,1,2,0,19.008467,0.354975
4,11001-50C-1129821-00017-00125-2023,125650000.0,2023,0,1,2,0,18.649011,0.145683


In [ ]:
# Información de los datos

datos_modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7512254 entries, 0 to 7512253
Data columns (total 9 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   PK                            object 
 1   VALOR                         float64
 2   YEAR_RADICA                   int64  
 3   PREDIOS_NUEVOS                int64  
 4   COUNT_DE                      int64  
 5   COUNT_A                       int64  
 6   FECHA_APERTURA_INCONSISTENTE  int64  
 7   VALOR_LOG                     float64
 8   Z_SCORE_RELATIVO              float64
dtypes: float64(3), int64(5), object(1)
memory usage: 515.8+ MB


# Enfoque con inteligencia artificial

## Alimentar el modelo

In [34]:
# Se obtienen las características para alimentar el modelo

features_modelo = datos_modelo.columns.tolist()
features_modelo.remove('PK')
features_modelo.remove('VALOR')

Para definir el porcentaje de contaminación del modelo revisamos cuántos valores con Z_SCORE_RELATIVO hay por encima de 3.5 o debajo de -3.5 (lo cual en estadística es raro que pase), y dividimos esta cantidad entre el total de datos.

In [38]:
conteo_z_extremo = sum(datos_modelo['Z_SCORE_RELATIVO'] > 3.5) + sum(datos_modelo['Z_SCORE_RELATIVO'] < -3.5)

porcentaje_contaminacion = conteo_z_extremo / datos_modelo.shape[0]
print(f"Porcentaje de contaminación estimado: {porcentaje_contaminacion * 100:.4f}%")

Porcentaje de contaminación estimado: 0.4174%


In [36]:
print("Iniciando entrenamiento del Isolation Forest...")

iso_forest = IsolationForest(
    n_estimators=100, 
    contamination=porcentaje_contaminacion,
    max_samples='auto',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

iso_forest.fit(datos_modelo[features_modelo])

Iniciando entrenamiento del Isolation Forest...


[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    3.8s remaining:   19.4s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    4.1s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   12.9s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   26.3s finished


,n_estimators,100
,max_samples,'auto'
,contamination,0.0041742465044446045
,max_features,1.0
,bootstrap,False
,n_jobs,-1
,random_state,42
,verbose,1
,warm_start,False


In [39]:
print("Calculando scores y etiquetas de anomalías...")

datos_modelo['IF_LABEL'] = iso_forest.predict(datos_modelo[features_modelo])
datos_modelo['IF_SCORE'] = iso_forest.decision_function(datos_modelo[features_modelo])

Calculando scores y etiquetas de anomalías...


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   12.9s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   26.5s finished
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:   13.3s
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   26.8s finished


In [40]:
# Resultados del modelo, los valores de IF_LABEL son -1 para anomalías y 1 para normales

datos_modelo['IF_LABEL'].value_counts()

IF_LABEL
 1    7480896
-1      31358
Name: count, dtype: int64

In [41]:
cond_anomalia = datos_modelo['IF_LABEL'] == -1

In [42]:
# Estadísticas de las transacciones marcadas como anómalas

datos_modelo[cond_anomalia].describe()

,VALOR,YEAR_RADICA,PREDIOS_NUEVOS,COUNT_DE,COUNT_A,FECHA_APERTURA_INCONSISTENTE,VALOR_LOG,Z_SCORE_RELATIVO,IF_LABEL,IF_SCORE
count,3.135800e+04,31358.000000,31358.000000,31358.000000,31358.000000,31358.000000,31358.000000,31358.000000,31358.0,31358.000000
mean,2.059408e+10,2019.762294,0.727119,3.256649,3.797500,0.689489,14.411661,-1.238586,-1.0,-0.030549
std,2.879537e+12,2.936933,0.445447,5.569452,11.725018,0.462710,5.109722,2.765534,0.0,0.026631
min,1.000000e-02,2015.000000,0.000000,0.000000,0.000000,0.000000,0.009950,-11.863091,-1.0,-0.127529
25%,6.244000e+04,2017.000000,0.000000,1.000000,1.000000,0.000000,11.041977,-3.051495,-1.0,-0.045030
50%,5.000000e+05,2021.000000,1.000000,1.000000,1.000000,1.000000,13.122365,-1.684032,-1.0,-0.022201
75%,3.500000e+08,2022.000000,1.000000,3.000000,3.000000,1.000000,19.673444,1.606324,-1.0,-0.009759
max,5.000000e+14,2023.000000,1.000000,278.000000,828.000000,1.000000,33.845629,12.068409,-1.0,-0.000001


Se tienen varias observaciones:

- En la columna VALOR se observan valores extremos como 0.01 pesos hasta una transacción con 500 billones, lo cual tiene sentido que sean anomalías ya que son valores muy extremos fuera de lo normal.

- La media del valor de las transacciones marcadas como anómalas es de 20'000 millones, valores muy altos para una transacción inmobiliaria normal.

- El 72.71% de las transacciones marcadas como anómalas son de predios nuevos, por lo que es posible que se estén usando predios nuevos para cometer algún tipo de fraude fiscal.

- La media de otorgantes y receptores está por encima de 3 cada uno, lo que significa que en la transacción anómala participan 6 o más personas en promedio, lo que puede indicar un posible loteo ilegal.

- El 68.95% tienen una fecha de apertura inconsistente, otro posible indicador de anomalías.

- Z-Score relativos extremos con valores desde -11.86 hasta 12.06, lo cual es un indicador de anomalías ya que en estadística los valores de Z-score por encima de 3.5 o debajo de -3.5 son escasos.

In [43]:
# Estadísticas de las transacciones marcadas como normales

datos_modelo[~cond_anomalia].describe()

,VALOR,YEAR_RADICA,PREDIOS_NUEVOS,COUNT_DE,COUNT_A,FECHA_APERTURA_INCONSISTENTE,VALOR_LOG,Z_SCORE_RELATIVO,IF_LABEL,IF_SCORE
count,7.480896e+06,7.480896e+06,7.480896e+06,7.480896e+06,7.480896e+06,7.480896e+06,7.480896e+06,7.480896e+06,7480896.0,7.480896e+06
mean,1.335039e+09,2.019802e+03,1.605142e-02,1.283786e+00,1.409441e+00,6.143491e-01,1.701397e+01,5.191834e-03,1.0,2.248384e-01
std,1.822211e+12,2.661630e+00,1.256733e-01,1.058944e+00,1.417019e+00,4.867487e-01,1.932073e+00,9.826078e-01,0.0,5.352792e-02
min,1.200000e-01,2.015000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.133287e-01,-1.251370e+01,1.0,5.765103e-09
25%,6.979000e+06,2.017000e+03,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.575842e+01,-6.037830e-01,1.0,2.055657e-01
50%,3.000000e+07,2.021000e+03,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.721671e+01,1.541236e-01,1.0,2.395273e-01
75%,1.000000e+08,2.022000e+03,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.842068e+01,6.927791e-01,1.0,2.632121e-01
max,3.790000e+15,2.023000e+03,1.000000e+00,3.020000e+02,1.304000e+03,1.000000e+00,3.587114e+01,1.216523e+01,1.0,2.862627e-01


Se tienen varias observaciones:

- En la columna VALOR se observan valores extremos como 0.12 pesos hasta una transacción con 3'790 billones, lo cual debería ser una anomalía, pero el modelo no fue capaz de detectarlo.

- La media del valor de las transacciones marcadas como normales es de 1'335 millones, un valor alto pero que puede llegar a ser común en algunas zonas del país. También se debe tener en cuenta que esta media está sesgada por el valor extremo de 3'790 billones.

- El 1.61% de las transacciones marcadas como normales son de predios nuevos.

- La media de otorgantes y receptores está por encima de 1 cada uno, lo que significa que en las transacciones normales en promedio participan 2 o 3 personas.

- El 6.14% tienen una fecha de apertura inconsistente.

- Z-Score relativos extremos con valores desde -12.51 hasta 12.17, lo cual es extraño en una transacción normal, por lo que es posible que el modelo no haya sido capaz de detectarlo.

# Enfoque con reglas de negocio

Aquí se definirán reglas de los límites de valores para considerar una transacción como normal.

In [51]:
limite_z_score = 3.5 # Límite para considerar una transacción como anómala según Z-Score en estadística

limite_valor_alto = datos_modelo['VALOR'].quantile(0.999) # Percentil 999 de los datos
print(f"Límite valor alto (percentil 99.9): {limite_valor_alto}")

limite_valor_bajo = datos_modelo['VALOR'].quantile(0.01)  # Percentil 1
print(f"Límite valor bajo (percentil 1): {limite_valor_bajo}")

Límite valor alto (percentil 99.9): 11912566800.0
Límite valor bajo (percentil 1): 100000.0


Los límites superiores e inferiores se obtuvieron calculando los percentiles 99.9 y 1 respectivamente, obteniendo $11'912.566.800 y $100.000, lo cual son valores aptos para umbrales de anomalías. 

En Colombia existen predios con valores superiores a los $10'000.000.000, pero sus transacciones no son muy comunes. Por otra parte, una transacción inmobiliaria de $100.000 es muy raro y es posible que sean transacciones ficticias para evadir impuestos y ocultar ganancias reales de una transacción.

In [52]:
# Definimos la función para clasificar las transacciones según reglas de negocio

def categorizar_transaccion(row):
    etiquetas = []
    
    # --- REGLA 1: ANOMALÍA FINANCIERA (PRIORIDAD ALTA) ---
    # Captura los "Billonarios" que el modelo ignoró y los "Regalados"
    if row['Z_SCORE_RELATIVO'] > limite_z_score:
        etiquetas.append("Sobreprecio extremo")
    elif row['Z_SCORE_RELATIVO'] < -limite_z_score:
        etiquetas.append("Subvaloración extrema")
        
    if row['VALOR'] > limite_valor_alto:
        etiquetas.append("Valor exorbitante")

    # --- REGLA 2: ANOMALÍA DE PATRÓN (INTELIGENCIA ARTIFICIAL) ---
    # Usamos el Isolation Forest para lo que NO es obvio por precio
    if row['IF_LABEL'] == -1:
        # Solo agregamos la etiqueta de IA si NO es ya una anomalía de precio obvia
        # para darle crédito al modelo por encontrar cosas "nuevas"
        if len(etiquetas) == 0: 
            etiquetas.append("Patrón atípico detectado por IA")
        else:
            # Si ya tenía etiqueta, es una confirmación doble
            etiquetas.append("Confirmado por IA")

    # Resultado Final
    if len(etiquetas) > 0:
        return " | ".join(etiquetas)
    else:
        return "Normal"

In [53]:
print("Clasificando anomalías...")
datos_modelo['DETALLE_ANOMALIA'] = datos_modelo.apply(categorizar_transaccion, axis=1)

Clasificando anomalías...


In [56]:
# Creamos una bandera binaria final para facilitar el filtrado
datos_modelo['ES_ANOMALO_FINAL'] = np.where(datos_modelo['DETALLE_ANOMALIA'] == "Normal", 0, 1)

In [69]:
# Vemos el resumen del éxito
print('\n--- RESUMEN DE HALLAZGOS ---')
print(datos_modelo['DETALLE_ANOMALIA'].value_counts())
print('-' * 30)
print('Porcentaje de transacciones con valor anómalas: ')
print(f"{datos_modelo['ES_ANOMALO_FINAL'].mean() * 100:.4f}%")


--- RESUMEN DE HALLAZGOS ---
DETALLE_ANOMALIA
Normal                                                         7453778
Patrón atípico detectado por IA                                  24471
Subvaloración extrema                                            21373
Subvaloración extrema | Confirmado por IA                         6315
Valor exorbitante                                                 2451
Sobreprecio extremo | Valor exorbitante                           1826
Sobreprecio extremo                                               1468
Sobreprecio extremo | Valor exorbitante | Confirmado por IA        200
Valor exorbitante | Confirmado por IA                              196
Sobreprecio extremo | Confirmado por IA                            176
Name: count, dtype: int64
------------------------------
Porcentaje de transacciones con valor anómalas: 
0.7784%


In [70]:
datos_modelo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7512254 entries, 0 to 7512253
Data columns (total 13 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   PK                            object 
 1   VALOR                         float64
 2   YEAR_RADICA                   int64  
 3   PREDIOS_NUEVOS                int64  
 4   COUNT_DE                      int64  
 5   COUNT_A                       int64  
 6   FECHA_APERTURA_INCONSISTENTE  int64  
 7   VALOR_LOG                     float64
 8   Z_SCORE_RELATIVO              float64
 9   IF_LABEL                      int64  
 10  IF_SCORE                      float64
 11  DETALLE_ANOMALIA              object 
 12  ES_ANOMALO_FINAL              int64  
dtypes: float64(4), int64(7), object(2)
memory usage: 745.1+ MB


In [68]:
# Guardamos los resultados en un nuevo archivo Parquet para análisis en Power BI

datos_con_anomalias = datos_modelo[datos_modelo['ES_ANOMALO_FINAL'] == 1]
datos_con_anomalias = datos_con_anomalias[['PK', 'ES_ANOMALO_FINAL', 'DETALLE_ANOMALIA']]
datos_con_anomalias.to_parquet('transacciones_anomalas_valor.parquet', index=False)